<center><h1>Полный гайд на математику оружия в Elden Ring</h1></center>

На просторах русскоязычного интернета много разной информации о том, как скалируется оружие в Elden Ring. Не скажу, что они плохие, даже есть очень хорошие. Но мне не хватало деталей в них, потому я сам решил разобраться и поделиться с вами моими находками <br>

Будем разбирать на примере **Cold Dagger +5** <br>
[![image.webp](https://i.postimg.cc/dtYBtvnp/image.webp)](https://postimg.cc/dD5d4bbj)

**Статы предмета**:
- BasePhysical: 71
- BaseMagic: 60
- Strenght Scaling: 31 -> **D**
- Dextirity Scaling: 58 -> **D**
- Intelligence Scaling: 52 -> **D**

 [![Screenshot-11.png](https://i.postimg.cc/6340TpkJ/Screenshot-11.png)](https://postimg.cc/62wRmt6Y)

## Ранжирование

- **E:** 0 - 24
- **D:** 25 - 59
- **C:** 60 - 89
- **B:** 90 - 139
- **A:** 140 - 174
- **S:** 175+


In [28]:
StrenghtScale = 31
DextirityScale  = 58
IntelligenceScale  = 52

BasePhysical = 71
BaseMagic = 60

#### В чем суть?

Расчет финального значения урона оружия зависит от 3 понятий: **Базовая атака**, **Скалирование параметра характеристики от оружия**, **Бонус от параметра характеристики**.

**[базовый урон] * ([скалирование оружия от характеристики] * [бонус от параметра характеристики])**

Если с **Базовой атакой** все достаточно легко. То другие два требуют пояснения.

**Скалирование параметра характеристики от оружия** - если зайти код игры, то мы можем увидеть, что на каждом уровне оружия есть свои коэффиценты, которы отвечают за повышение базовой атаки и **Скалирование параметра характеристики от оружия**
они выглядат так:
<br>[![Screenshot-7.png](https://i.postimg.cc/44tPgnvp/Screenshot-7.png)](https://postimg.cc/YjrYNrsS)<br>
[![Screenshot-8.png](https://i.postimg.cc/WzR7ZPVD/Screenshot-8.png)](https://postimg.cc/jwXNVmLK)<br>
Строки **Correction %** как раз и отвечают за склаирование параметра. По сути, мы наш базовой параметр скалирования(например сила 31) тут умножим на 1.250 и получим 38.75. Это будет нашим скалированием.<br><br>
Строки **Damage %** делают тоже самое, только с базовой атакой оружия, в нашем случае с физикой: 71 * 1.04 = 73.84

Ну круто, уже основные значения для расчетов посчитаны, напишем их в коде

In [29]:
scale_str = scale_dex = scale_int = 1.25
scale_phys_damage = scale_magic_damage = 1.04

PhysicalBaseDamage = BasePhysical * scale_phys_damage
MagicBaseDamage = BaseMagic * scale_magic_damage

ScaleFromStrenght = StrenghtScale  * scale_str
ScaleFromDextirity = DextirityScale  * scale_dex
ScaleFromIntel = IntelligenceScale  * scale_int


print(f"""
Физический базовый урон: {PhysicalBaseDamage:.2f}
Магический базовый урон: {MagicBaseDamage:.2f}

Увеличение от Силы на: {ScaleFromStrenght:.2f}% также D
Увеличение от Ловкости на: {ScaleFromDextirity:.2f}% теперь это С 
Увеличение от Интеллекта на: {ScaleFromIntel:.2f}% теперь это С 
""")


Физический базовый урон: 73.84
Магический базовый урон: 62.40

Увеличение от Силы на: 38.75% также D
Увеличение от Ловкости на: 72.50% теперь это С 
Увеличение от Интеллекта на: 65.00% теперь это С 



### Бонус от параметра характеристики
Настало для самого сложного. Тут уже без вдумчивости, если вы сталкиваетесь впервые с таким - не обойтись. Напомню, что финальное значение будет считаться как:
 
**[базовый урон] * ([скалирование оружия от характеристики] * [бонус от параметра характеристики])**

В нашем случае так как у нашего **Cold Dagger +5** есть скейлы от силы, ловкости, интеллекта, то итоговая формула будет выглядеть так:

**[базовый урон] * ([скалирование оружия от силы] * [бонус от параметра силы] + [скалирование от лвк] * [бонус от параметра лвк] + [скалирование от инт] * [бонус от параметра инт])**

Если **[базовый урон]** и **[скалирование оружия от силы]** мы считали просто как умножение базы на коэффицент, то расчет **[бонус от параметра силы]** имеет более сложную структуру, которая и будет влиять на основной баланс в игре.

Сейчас я введу понятия, а затем покажу основную формулу.
***
#### Понятия:

Стоит оговорится, что значения этих параметров, варируется от оружия к оружию.
[![Screenshot-12.png](https://i.postimg.cc/Ghxj4N52/Screenshot-12.png)](https://postimg.cc/9rz9SxV5)

 
**Точки смены поведения (Breakpoints)** - Они представляют собой ключевые точки развития игрока. Допустим, если у тебя Сила 19, то ты будешь между Stat1 и Stat2. Где твоё минимальное значение 18, а максимум 60.
Также это можно интерпертировать, как стадия развития персонажа в игре.
- Stat 0: 1
- Stat 1: 18
- Stat 2: 60
- Stat 3: 80
- Stat 4: 150

Отсюда<br>
- Stat 0: Начальное значение.<br>
- Stat 1-3: Мягкие пределы, после которых рост статистики начинает замедляться.<br>
- Stat 4: Конечное значение или максимальный уровень статистики.<br>
<br><br>

**Рост (Growth)** - значения роста показывают, насколько увеличится характеристика персонажа на каждом этапе. Рост 0 означает отсутсвие увеличения, а на последующих уровнях рост постепенно увеличивается, достигая максимума на уровне 4.

- Growth 0: 0
- Growth 1: 25
- Growth 2: 75
- Growth 3: 90
- Growth 4: 110<br><br>

**Экспоненты (Exponent)** -  как быстро статистика достигает своих значений роста. Экспонента 1 означает линейный рост, тогда как значения выше или ниже 1 указывают на нелинейный рост.
- Exponent 0: 1.2 - Положительное значение указывает на более быстрое увеличение в начале и замедление к концу.
- Exponent 1: -1.2 - Отрицательное значение указывает на уменьшение роста
- Exponent 2: 1 - Линейный рост, что означает равномерное увеличение характеристик на этих уровнях.
- Exponent 3: 1
- Exponent 4: 1
***

#### Интуитивно можно попробовать понять из этих картинок:
[![Screenshot-9.png](https://i.postimg.cc/hv91b342/Screenshot-9.png)](https://postimg.cc/y38R7jZR)
[![Screenshot-10.png](https://i.postimg.cc/kgrBdXcR/Screenshot-10.png)](https://postimg.cc/SJry64Hm)

На графике видно что Stat это ось X и те значения из таблицы это точка измененения поведение функции

adjPt это наша экспонента, которая как раз и меняет поведение функции
***
### Формула:

Для расчета бонуса используется след формула:
```
Ratio = (INPUT - StatMIN) / (StatMAX - StatMIN)

If ExponentMIN > 0
    Growth = Ratio^ExponentMIN
If ExponentMIN < 0
    Growth = 1 - [(1 - Ratio)^|ExponentMIN|]
OUTPUT = GrowMIN + [(GrowMAX - GrowMIN) * Growth]
```

Разберем это на части, предположим, у игрока 14 силы, 14 ловкости и 40 интеллекта. <br>

Просматривая Stat 0 - Stat 4 в таблице  выше, мы видим, что 14 попадает между Stat 0 и Stat 1, а 40 попадает между Stat 1 и Stat 2. Итак, в формуле для силы и ловкости все экземпляры MIN и MAX будут значениями 0 и 1, а для интеллекта - значениями 1 и 2. Вот формула для обоих с нашими данными:

### Для силы (Str) и ловкости (Dex) т.к оба 14
```
Ratio = (14 - 1) / (18 - 1)

If 1.2 > 0
    Growth = (13/17)^1.2
If 1.2 < 0
    Growth = 1 - [(1 - (13/17))^1.2]

OUTPUT = 0 + [(25 - 0) * ((13/17)^1.2)] = ~18.11896%
```

### Для интеллекта (Int)
```
Ratio = (40 - 18) / (60 - 18)

If -1.2 > 0:
    Growth = (22/42)^-1.2
If -1.2 < 0:
    Growth = 1 - [(1 - (22/42))^1.2]

OUTPUT = 25 + [(75 - 25) * ~0.589] = ~54.47388%
```

In [30]:
def calculate_ratio(input_stat, stat_min, stat_max):
    return (input_stat - stat_min) / (stat_max - stat_min)

def calculate_growth(ratio, exponent_min):
    if exponent_min > 0:
        return ratio ** exponent_min
    elif exponent_min < 0:
        return 1 - ((1 - ratio) ** abs(exponent_min))
    else:
        return 1
    
def calculate_output(grow_min, grow_max, growth):
    return grow_min + (grow_max - grow_min) * growth

str_char = dex_char = 14 # сила и ловкость

STATS = [1, 18, 60, 80, 150]

# Инициализируем STAT_MIN и STAT_MAX
STAT_MIN = None
STAT_MAX = None
index_min = None
index_max = None
# Находим STAT_MIN и STAT_MAX
for i in range(len(STATS) - 1):
    if STATS[i] <= str_char < STATS[i + 1]:
        STAT_MIN = STATS[i]
        index_min = i
        STAT_MAX = STATS[i + 1]
        index_max = i + 1
        break

if STAT_MIN is not None and STAT_MAX is not None:
    print(f"Для значения {str_char} диапазон: [{STAT_MIN}, {STAT_MAX}]")

phys_ratio = calculate_ratio(str_char, STAT_MIN, STAT_MAX) # for str and dex
print(f"Значение {phys_ratio} является нашим RATIO для силы и ловкости")


Для значения 14 диапазон: [1, 18]
Значение 0.7647058823529411 является нашим RATIO для силы и ловкости


In [31]:
GROWTH = [0, 25, 75, 90, 110]
EXPONENTS = [1.2, -1.2, 1, 1, 1]

phys_growth = calculate_growth(phys_ratio, EXPONENTS[index_min])
print(f"Значение {phys_growth} является нашим показателем роста для силы и ловкости")

phys_bonus = calculate_output(GROWTH[index_min],GROWTH[index_max], phys_growth)
print(f"Значение {phys_bonus}% является нашим бонуса от силы и ловкости")


Значение 0.7247584993864192 является нашим показателем роста для силы и ловкости
Значение 18.118962484660482% является нашим бонуса от силы и ловкости


In [32]:
# Тоже самое для магии

int_char = 40 # интеллект

STAT_MIN = None
STAT_MAX = None
index_min = None
index_max = None

for i in range(len(STATS) - 1):
    if STATS[i] <= int_char < STATS[i + 1]:
        STAT_MIN = STATS[i]
        index_min = i
        STAT_MAX = STATS[i + 1]
        index_max = i + 1
        break

if STAT_MIN is not None and STAT_MAX is not None:
    print(f"Для значения {int_char} диапазон: [{STAT_MIN}, {STAT_MAX}]")

int_ratio = calculate_ratio(int_char, STAT_MIN, STAT_MAX) # for int
print(f"Значение {int_ratio} является нашим RATIO для интеллекта")

int_growth = calculate_growth(int_ratio, EXPONENTS[index_min])
print(f"Значение {int_growth} является нашим показателем роста для интеллекта")

int_bonus = calculate_output(GROWTH[index_min],GROWTH[index_max], int_growth)
print(f"Значение {int_bonus}% является нашим бонуса от интеллекта")


Для значения 40 диапазон: [18, 60]
Значение 0.5238095238095238 является нашим RATIO для интеллекта
Значение 0.5894776121914185 является нашим показателем роста для интеллекта
Значение 54.47388060957093% является нашим бонуса от интеллекта


In [33]:
int_bonus = calculate_output(GROWTH[index_min],GROWTH[index_max], int_growth)
print(f"Значение {int_bonus}% является нашим бонуса от интеллекта")
phys_bonus = calculate_output(GROWTH[index_min],GROWTH[index_max], phys_growth)
print(f"Значение {phys_bonus}% является нашим бонуса от силы и ловкости")

Значение 54.47388060957093% является нашим бонуса от интеллекта
Значение 61.237924969320964% является нашим бонуса от силы и ловкости


<center><h2>Итоговый расчет</h2></center>

Напоминаю, нам надо найти:

**[базовый урон] * ([скалирование оружия от силы] * [бонус от параметра силы] + [скалирование от лвк] * [бонус от параметра лвк] + [скалирование от инт] * [бонус от параметра инт])**

Только **важно** перевести наши скалирование и бонусы из % в числа. Просто поделим их на 100.

In [34]:
int_bonus /= 100
phys_bonus /= 100

ScaleFromStrenght /= 100
ScaleFromDextirity /= 100
ScaleFromIntel /= 100

In [35]:
print(f"""
Значения после деления на 100:
      
Бонус от интеллекта: {int_bonus:.2f}
Физический бонус: {phys_bonus:.2f}

Увеличение от Силы: {ScaleFromStrenght:.2f}
Увеличение от Ловкости: {ScaleFromDextirity:.2f}
Увеличение от Интеллекта: {ScaleFromIntel:.2f}
""")


Значения после деления на 100:
      
Бонус от интеллекта: 0.54
Физический бонус: 0.61

Увеличение от Силы: 0.39
Увеличение от Ловкости: 0.72
Увеличение от Интеллекта: 0.65



In [36]:
# Считаем
import math 

# Считаем физический урон
ForStrength = PhysicalBaseDamage * ScaleFromStrenght * phys_bonus
ForDextirity = PhysicalBaseDamage * ScaleFromDextirity * phys_bonus
FinalPhysDamage = PhysicalBaseDamage + ForStrength + ForDextirity

# Считаем магический урон
ForMagic = MagicBaseDamage * ScaleFromIntel * int_bonus
FinalIntDamage = MagicBaseDamage + ForMagic
# Окончательный урон
FinalSumDamage = math.floor(FinalPhysDamage + FinalIntDamage) # округляем в меньшую строну, т.к так происходит в игре


print(f"""
Значения после деления на 100:
Бонус от интеллекта: {int_bonus:.2f}
Физический бонус: {phys_bonus:.2f}

Увеличение от Силы: {ScaleFromStrenght:.2f}
Увеличение от Ловкости: {ScaleFromDextirity:.2f}
Увеличение от Интеллекта: {ScaleFromIntel:.2f}

Расчеты урона:
Базовый физический урон: {PhysicalBaseDamage:.2f}
Базовый магический урон: {MagicBaseDamage:.2f}

Физический урон от силы: {ForStrength:.2f}
Физический урон от ловкости: {ForDextirity:.2f}
Итоговый физический урон: {FinalPhysDamage:.2f}

Магический урон от интеллекта: {ForMagic:.2f}
Итоговый магический урон: {FinalIntDamage:.2f}

Итоговый суммарный урон: {FinalSumDamage}
""")


Значения после деления на 100:
Бонус от интеллекта: 0.54
Физический бонус: 0.61

Увеличение от Силы: 0.39
Увеличение от Ловкости: 0.72
Увеличение от Интеллекта: 0.65

Расчеты урона:
Базовый физический урон: 73.84
Базовый магический урон: 62.40

Физический урон от силы: 17.52
Физический урон от ловкости: 32.78
Итоговый физический урон: 124.15

Магический урон от интеллекта: 22.09
Итоговый магический урон: 84.49

Итоговый суммарный урон: 208

